# Check if representative group knockoffs are working

In [5]:
# load packages for this tutorial
using Revise
using Knockoffs
using LinearAlgebra
using Random
using StatsBase
using Statistics
using ToeplitzMatrices
using Distributions
using Clustering
using ProgressMeter
using LowRankApprox
using Plots
using CSV, DataFrames
gr(fmt=:png);

# some helper functions to compute power and empirical FDR
function TP(correct_groups, signif_groups)
    return length(signif_groups ∩ correct_groups) / max(1, length(correct_groups))
end
function TP(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return TP(correct_groups, signif_groups)
end
function power(correct_snps, discovered_snps)
    return length(discovered_snps ∩ correct_snps) / length(correct_snps)
end
function FDR(correct_groups, signif_groups)
    FP = length(signif_groups) - length(signif_groups ∩ correct_groups) # number of false positives
    return FP / max(1, length(signif_groups))
end
function FDR(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return FDR(correct_groups, signif_groups)
end
function get_signif_groups(β, groups)
    correct_groups = Int[]
    for i in findall(!iszero, β)
        g = groups[i]
        g ∈ correct_groups || push!(correct_groups, g)
    end
    return correct_groups
end

get_signif_groups (generic function with 1 method)

## gnomdAD panel

In [169]:
datadir = "/Users/biona001/Benjamin_Folder/research/4th_project/group_knockoff_test_data"
covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame)
Σ = covfile |> Matrix{Float64}
Σ = 0.99Σ + 0.01I #ensure PSD

# test on smaller data
idx = 120 # 1261 # includes largest group with 192 members
Σ = Σ[1:idx, 1:idx];

In [302]:
# simulate data
m = 1
p = size(Σ, 1)
k = 10 # number of causal groups
n = 2250 # sample size

# simulate X
μ = zeros(p)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups empirically
nrep = 5
groups, group_reps = hc_partition_groups(X, rep_method=:rss, nrep=nrep, force_contiguous=true)
unique_groups = unique(groups)
countmap(groups) |> values |> collect |> sort

# rep group knockoffs
# @time ko = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)
@time Xr, Xc, X̃r, X̃c, X̃r_correct, X̃c_correct = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)

size(S) = (45, 45)
size(X̃new) = (2250, 120)
size(X̃r) = (2250, 45)
size(X̃c) = (2250, 75)
  0.049497 seconds (1.80 k allocations: 27.915 MiB)


([1.6300273821086542 -0.13103797352308488 … -0.1273761444444048 -0.34924204183999485; 0.3805709120831205 0.17461735249439253 … 0.7639420392439948 0.7460130251681191; … ; -0.5025873111458099 2.701036972537028 … -0.23670808896727213 -1.7823346410578118; 1.1568830797383602 0.40610739484142055 … -0.04461080465884314 -0.7846148095816645], [-0.20152088333115456 -0.20650098005510775 … -1.103930798258196 -1.1594966684668362; -0.160323513507529 -1.1558632692200557 … 1.1420179959175518 0.601815865209059; … ; 1.9318264855217038 1.9288499737126252 … -0.6537726988626349 -0.7042383957849201; 1.4644273299045694 0.3806038995938459 … 0.39499668844636165 0.6634182653911984], [0.7791787431995464 -1.000753667860982 … 0.6978903531680367 -2.9805739809095435; 2.4408666549277926 -2.9160195071731647 … -0.5020135761997957 -0.02773354282201046; … ; 1.6060818584271113 -1.951176584768308 … 0.15412550128634692 -3.322695118705263; -2.290215415310235 2.8903684509827396 … 1.1091378500344704 2.4137356730957427], [-18.8

In [298]:
cor(Xr)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         0.0113552   0.344703   …   0.110612    0.117837    0.150459
  0.0113552   1.0         0.0878911      0.509606    0.510395    0.335019
  0.344703    0.0878911   1.0            0.278089    0.282071    0.36037
 -0.0939312  -0.0637252  -0.0848497     -0.066723   -0.0680524  -0.0435355
  0.0443258  -0.0357064   0.0750953     -0.0513268  -0.050556   -0.0375401
 -0.123467   -0.0572152  -0.0665421  …  -0.0812474  -0.0825897  -0.0291871
  0.113424    0.506083    0.279623       0.98215     0.980966    0.725485
  0.110612    0.509606    0.278089       1.0         0.987808    0.726371
  0.117837    0.510395    0.282071       0.987808    1.0         0.725505
  0.150459    0.335019    0.36037        0.726371    0.725505    1.0

In [299]:
cor(X̃r_correct)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0        -0.0469187   0.320928   …   0.107857    0.109635    0.142476
 -0.0469187   1.0         0.0686697      0.4854      0.48787     0.318639
  0.320928    0.0686697   1.0            0.250207    0.254169    0.343777
 -0.121054   -0.035295   -0.0846095     -0.0608074  -0.0586042  -0.0644673
  0.0589452  -0.0326455   0.103702      -0.0331175  -0.0282574  -0.0283155
 -0.0828774  -0.0294553  -0.0527228  …  -0.0406056  -0.0446366  -0.0200239
  0.107249    0.490101    0.252113       0.981982    0.980152    0.723458
  0.107857    0.4854      0.250207       1.0         0.98838     0.725135
  0.109635    0.48787     0.254169       0.98838     1.0         0.726457
  0.142476    0.318639    0.343777       0.725135    0.726457    1.0

In [300]:
cor(Xc)[1:10, 1:10]

10×10 Matrix{Float64}:
 1.0       0.790559  0.796962  0.793148  …  0.787267  0.629009  0.60204
 0.790559  1.0       0.97263   0.964453     0.957958  0.773607  0.756972
 0.796962  0.97263   1.0       0.976812     0.970603  0.785816  0.766135
 0.793148  0.964453  0.976812  1.0          0.96567   0.783295  0.761316
 0.799049  0.971518  0.976256  0.970826     0.966519  0.777273  0.757067
 0.784374  0.956161  0.965952  0.961368  …  0.957598  0.771488  0.751689
 0.794808  0.970314  0.981221  0.974843     0.972961  0.78501   0.765031
 0.787267  0.957958  0.970603  0.96567      1.0       0.78525   0.763969
 0.629009  0.773607  0.785816  0.783295     0.78525   1.0       0.948028
 0.60204   0.756972  0.766135  0.761316     0.763969  0.948028  1.0

In [303]:
cor(X̃c_correct)[1:10, 1:10]

10×10 Matrix{Float64}:
 1.0       0.793783  0.789148  0.784424  …  0.777297  0.609031  0.59219
 0.793783  1.0       0.97219   0.967286     0.958805  0.76371   0.741382
 0.789148  0.97219   1.0       0.977908     0.97062   0.772893  0.752297
 0.784424  0.967286  0.977908  1.0          0.966506  0.768546  0.746543
 0.794478  0.973214  0.976904  0.972866     0.965329  0.765975  0.746169
 0.792156  0.972135  0.983457  0.977356  …  0.972771  0.772662  0.753506
 0.777574  0.957551  0.966885  0.963889     0.956986  0.75512   0.737605
 0.777297  0.958805  0.97062   0.966506     1.0       0.76373   0.741761
 0.609031  0.76371   0.772893  0.768546     0.76373   1.0       0.947337
 0.59219   0.741382  0.752297  0.746543     0.741761  0.947337  1.0

In [194]:
# simulate data
m = 1
p = size(Σ, 1)
k = 10 # number of causal groups
n = 2250 # sample size

# simulate X
μ = zeros(p)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups empirically
nrep = 1
groups, group_reps = id_partition_groups(X, rep_method=:rss, nrep=nrep)
unique_groups = unique(groups)
countmap(groups) |> values |> collect |> sort

# rep group knockoffs
@time ko = modelX_gaussian_group_knockoffs(X, :maxent, groups, μ, Σ, m=m);

  0.189736 seconds (14.79 k allocations: 17.020 MiB)


In [182]:
cor(X)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         0.0136149   0.343179   …   0.117435    0.11872     0.124917
  0.0136149   1.0         0.0805718      0.503197    0.504936    0.322599
  0.343179    0.0805718   1.0            0.251345    0.25585     0.337278
 -0.132733   -0.0676995  -0.0636298     -0.0694679  -0.0693109  -0.0431528
  0.0807121  -0.0274369   0.110185      -0.072626   -0.0690193  -0.0710977
 -0.112806   -0.0316348  -0.0581683  …  -0.050345   -0.0469703  -0.0304699
  0.119676    0.50619     0.255603       0.981795    0.981342    0.716044
  0.117435    0.503197    0.251345       1.0         0.988406    0.714763
  0.11872     0.504936    0.25585        0.988406    1.0         0.71204
  0.124917    0.322599    0.337278       0.714763    0.71204     1.0

In [183]:
cor(ko.X̃)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0          -0.000374575  …   0.101817    0.100802    0.143131
 -0.000374575   1.0              0.463328    0.473686    0.30033
  0.315817      0.112863         0.234097    0.230312    0.312048
 -0.121171     -0.0274317       -0.0231393  -0.0198914  -0.0180876
  0.0562746    -0.0740926       -0.0732009  -0.0709649  -0.0446842
 -0.110251     -0.0582016    …  -0.0873943  -0.0826741  -0.0647018
  0.0914291     0.448818         0.969039    0.961999    0.683481
  0.101817      0.463328         1.0         0.985738    0.704234
  0.100802      0.473686         0.985738    1.0         0.706248
  0.143131      0.30033          0.704234    0.706248    1.0

## One simulation

In [22]:
# simulate data
m = 1
p = 500
k = 10 # number of causal groups
n = 250 # sample size
μ = zeros(p)
# Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

# define true groups and true covariance
groupsizes = [5 for i in 1:div(p, 5)] 
groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
Σ = simulate_block_covariance(groups, 0.75, 0.0)

# define groups empirically
nrep = 1
groups, group_reps = id_partition_groups(X, nrep=nrep)
unique_groups = unique(groups)

# simulate X
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# simulate βtrue: SNPs in causal groups will share the same effect 
δ = 2
βtrue = zeros(p)
shuffle!(unique_groups)
causal_groups = unique_groups[1:k]
for g in causal_groups
    β_idx = findall(x -> x == g, groups)
    effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
    βtrue[β_idx] .= effect_size
end

# simulate y
ϵ = randn(n)
y = X * βtrue + ϵ
correct_snps = findall(!iszero, βtrue)

# fully general me
me_t = @elapsed me = modelX_gaussian_group_knockoffs(
    X, :maxent, groups, μ, Σ, 
    m = m,
    niter = 10,
    tol = 0.01,    # convergence tolerance
    verbose=false, # whether to print informative intermediate results
)
me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_ssum = sum(abs.(me_ko_filter.ko.S))
@show me_power, me_fdr

# representative ME knockoffs
rme = modelX_gaussian_rep_group_knockoffs(
    X, :maxent, μ, Σ, groups, group_reps,
    m = m, 
    nrep = nrep
);
rme_ko_filter = fit_lasso(y, rme)
discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
rme_power = round(TP(causal_groups, discovered_groups), digits=3)
rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
@show rme_power, rme_fdr

(me_power, me_fdr) = (0.9, 0.1)
(rme_power, rme_fdr) = (1.0, 0.167)


(1.0, 0.167)

### Hierarchica clustering, target FDR = 0.1, nrep=2, m = 1

In [24]:
fdr_hat = 0.0
nsims = 10
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 1
    p = 500
    k = 10 # number of causal groups
    n = 1250 # sample size
    μ = zeros(p)
    # Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

    # define true groups and true covariance
    groupsizes = [5 for i in 1:div(p, 5)] 
    groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
    Σ = simulate_block_covariance(groups, 0.75, 0.0)

    # define groups empirically
    nrep = 2
    groups, group_reps = hc_partition_groups(Σ, nrep=nrep)
    unique_groups = unique(groups)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # simulate βtrue: SNPs in causal groups will share the same effect 
    δ = 1
    βtrue = zeros(p)
    shuffle!(unique_groups)
    causal_groups = unique_groups[1:k]
    for g in causal_groups
        β_idx = findall(x -> x == g, groups)
        effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
        βtrue[β_idx] .= effect_size
    end

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ
    correct_snps = findall(!iszero, βtrue)

    # fully general me
    me_t = @elapsed me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
    me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims

(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.231)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)


0.0686

### interpolative decomposition, target FDR = 0.1, nrep=2, m = 5

In [29]:
fdr_hat = 0.0
nsims = 10
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 5
    p = 500
    k = 10 # number of causal groups
    n = 1250 # sample size
    μ = zeros(p)
    # Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

    # define true groups and true covariance
    groupsizes = [5 for i in 1:div(p, 5)] 
    groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
    Σ = simulate_block_covariance(groups, 0.75, 0.0)

    # define groups empirically
    nrep = 2
    groups, group_reps = id_partition_groups(X, nrep=nrep)
    unique_groups = unique(groups)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # simulate βtrue: SNPs in causal groups will share the same effect 
    δ = 1
    βtrue = zeros(p)
    shuffle!(unique_groups)
    causal_groups = unique_groups[1:k]
    for g in causal_groups
        β_idx = findall(x -> x == g, groups)
        effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
        βtrue[β_idx] .= effect_size
    end

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ
    correct_snps = findall(!iszero, βtrue)

    # fully general me
    me_t = @elapsed me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
    me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims

(me_power, me_fdr) = (1.0, 0.167)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.286)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.167)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.167)


0.0774

### interpolative decomposition, target FDR = 0.25, nrep=5, m = 5

In [31]:
fdr_hat = 0.0
nsims = 20 # 10 showed slightly inflated FDR, try 20
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 5
    p = 500
    k = 10 # number of causal groups
    n = 1250 # sample size
    μ = zeros(p)
    # Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

    # define true groups and true covariance
    groupsizes = [5 for i in 1:div(p, 5)] 
    groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
    Σ = simulate_block_covariance(groups, 0.75, 0.0)

    # define groups empirically
    nrep = 5
    groups, group_reps = id_partition_groups(X, nrep=nrep)
    unique_groups = unique(groups)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # simulate βtrue: SNPs in causal groups will share the same effect 
    δ = 1
    βtrue = zeros(p)
    shuffle!(unique_groups)
    causal_groups = unique_groups[1:k]
    for g in causal_groups
        β_idx = findall(x -> x == g, groups)
        effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
        βtrue[β_idx] .= effect_size
    end

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ
    correct_snps = findall(!iszero, βtrue)

    # fully general me
    me_t = @elapsed me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
    me_power = round(TP(causal_groups, me_ko_filter.βs[4], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[4], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[4])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims

(me_power, me_fdr) = (1.0, 0.333)
(rme_power, rme_fdr) = (1.0, 0.524)
(me_power, me_fdr) = (1.0, 0.524)
(rme_power, rme_fdr) = (1.0, 0.412)
(me_power, me_fdr) = (1.0, 0.412)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.167)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.375)
(rme_power, rme_fdr) = (1.0, 0.333)
(me_power, me_fdr) = (1.0, 0.375)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.333)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.375)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.231)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.

0.22034999999999996